## Task 1

In [1]:
# Task 1
import re
import numpy as np

# load up the small dataset
corpus = sc.textFile("s3://chrisjermainebucket/comp330_A5/TrainingDataOneLinePerDoc.txt")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1637440549028_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# each entry in validLines will be a line from the text file
validLines = corpus.filter(lambda x: 'id' in x)
# now we transform it into a bunch of (docID, text) pairs
keyAndText = validLines.map(lambda x: (x[x.index('id="') + 4:x.index('" url=')], x[x.index('">') + 2:]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x: (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))

# now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey(lambda a, b: a + b)

# and get the top 20,000 words in a local array
# each entry is a ("word1", count) pair
topWords = allCounts.top(20000, lambda x: x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# and we'll create a RDD that has a bunch of (word, dictNum) pairs
# start by creating an RDD that has the number 0 thru 20000
# 20000 is the number of words that will be in our dictionary
twentyK = sc.parallelize(range(20000))

# now, we transform (0), (1), (2), ... to ("mostcommonword", 0) ("nextmostcommon", 1), ...
# the number will be the spot in the dictionary used to tell us where the word is located
# A bunch of (word, posInDictionary) pairs
dictionary = twentyK.map(lambda x: (topWords[x][0], x))

# Collect the Rdd to a Dict
localDict = dictionary.collectAsMap()
for inputWord in ["applicant", "and", "attack", "protein", "car"]:
    if inputWord in localDict:
        print(f'{inputWord}: {localDict[inputWord]}')
    else:
        print(f'{inputWord}: -1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

applicant: 448
and: 2
attack: 514
protein: 3167
car: 652

## Task 2

In [5]:
# Task 2
import math

# A bunch of (word, docID) pairs
wordDictPair = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))
# Join the two RDDs, you'll have a bunch of (word, (docID, posInDictionary)) pairs
wordPair = wordDictPair.join(dictionary)

# Get a bunch of (docid, (listOfAllDictonaryPos)) pairs
docIDIdxPair = wordPair.map(lambda x: (x[1][0], x[1][1]))
docIDAllIdxPair = docIDIdxPair.groupByKey()
docIDAllIdxList = docIDAllIdxPair.map(lambda x: (x[0], list(x[1])))

# Then finally, you will write a map () that will take that RDD and convert into the listOfAllDictonary
def transformNP(idxList):
    array = np.zeros(20000)
    for i in idxList:
        array[i] += 1
    return array
result = docIDAllIdxList.map(lambda x: (x[0], transformNP(x[1])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def findWord(wordList):
    arr = np.zeros(20000)
    for idx, i in enumerate(wordList):
        if i > 0:
            arr[idx] += 1
    return arr

# IDF
def IDF(result):
    cnt = 0
    val = result.map(lambda x: ("key", findWord(x[1])))
    finalList = val.reduceByKey(lambda a, b: a + b)
    return finalList

# Calculate IDF values
size = result.count()
IDFList = IDF(result)
IDFArr = np.array(IDFList.lookup("key"))
IDFArr = np.log(size / IDFArr)

# TF-IDF
def TFIDF(arrCnt):
    TFArr = arrCnt / arrCnt.sum()
    finalResult = np.multiply(TFArr, IDFArr)
    return finalResult

def predictLabel(inputStr):
    regex = re.compile('[^a-zA-Z]')

    # Input string to a bunch of words
    words = regex.sub(' ', inputStr).lower().split()
    wordArr = np.zeros(20000)

    for word in words:
        if word in localDict.keys():
            wordArr[localDict[word]] += 1
    inputTFIDF = TFIDF(wordArr)
    mean = np.mean(inputTFIDF)
    
    return inputTFIDF

# TFIDF for all dictionaries
trainingInfo = keyAndText.map(lambda x: (x[0], predictLabel(x[1])))
Data = keyAndText.map(lambda x: (1 if x[0][0:2] == "AU" else 0, predictLabel(x[1])))
trainingData = Data.map(lambda x: (x[0], x[1][0]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
meanVec = trainingData.map(lambda x: ("key", x[1]))
meanVec = meanVec.reduceByKey(lambda a, b: a+b)
meanVec = meanVec.map(lambda x: x[1])
mean = meanVec.collect()[0] / size

varVec = trainingData.map(lambda x: ("key", 10000 * np.square(x[1] - mean)))
varVec = varVec.reduceByKey(lambda a, b: a+b)
varVec = varVec.map(lambda x: x[1])
var = np.sqrt(varVec.collect()[0] / size)
var[var == 0] = 1
training = trainingData.map(lambda x: (x[0], (x[1] - mean) / var))
training = training.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
training.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, array([ 0.00289811,  0.00963589, -0.01407146, ..., -0.00037282,
       -0.00036121, -0.00022703]))]

In [9]:
# x is the data set
# y is the labels
# w is the current set of weights
# c is the weight of the slack variables

# Evaluates and returns the gradient
def gradient(train, w, c):
    l2 = np.linalg.norm(w, 2)
    grad_per_row = train.map(lambda x: -1 * x[0] * x[1] + x[1] * (np.exp(np.dot(x[1], w)) / (1 + np.exp(np.dot(x[1], w)))) + c * w * 2)
    acc = np.zeros(20000)
    grad = grad_per_row.fold(acc, lambda a, b: a + b)
    print(f"Gradient: {grad}")
    return grad

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from operator import add

# x is the data set
# y is the labels
# w is the current set of weights
# c is the weight of the slack variables

# Evaluates the loss function and returns the loss
def f(train, w, c):
    l2 = np.linalg.norm(w, 2)
    loss_per_row = train.map(lambda x: ("key", (-1 * x[0] * np.dot(x[1], w)) + np.log(1 + np.exp(np.dot(x[1], w))) + c * l2 * l2))
    loss = loss_per_row.reduceByKey(add).collect()
    loss = loss[0][1]
    print(f"Loss: {loss}")
    return loss

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# performs gradient descent optimization, returns the learned set of weights
# uses the bold driver to set the learning rate
#
# x is the data set
# y is the labels
# w is the current set of weights  to start with
# c is the weight of the slack variable
def gd_optimize(train, w, c):
    rate = 0.001
    w_last = w + np.full(20000, 1.0) 
    while (abs(f(train, w, c) - f(train, w_last, c))) > 10e-4:
        w_last = w 
        w = w - rate * gradient(train, w, c)
        if f(train, w, c) > f(train, w_last, c):
            rate = rate * 0.5
        else:
            rate = rate * 1.1
        print(f(train, w, c))
    return w

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
np.random.seed(42)
w = np.random.normal(0, 0.1, size=(20000))
c = 0.01
w = gd_optimize(training, w, 0.01)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Loss: 457010.70866251236
Loss: 34337859.40714984
Gradient: [ 141.26390343  -67.68750464  267.74549685 ...  129.13228396  577.85949739
 -546.74394342]
Loss: 2046941.6401948133
Loss: 457010.70866251236
Loss: 2046941.6401948133
2046941.6401948133
Loss: 2046941.6401948135
Loss: 457010.70866251236
Gradient: [ -336.57735512   161.69237857  -651.55062176 ...  -307.40409101
 -1375.25659548  1301.02141515]
Loss: 1038385.3825873439
Loss: 2046941.6401948133
Loss: 1038385.382587344
1038385.382587344
Loss: 1038385.3825873439
Loss: 2046941.6401948133
Gradient: [ 232.71070362 -112.6671571   467.67715483 ...  212.19356804  948.77166474
 -897.39474599]
Loss: 798689.721393297
Loss: 1038385.382587344
Loss: 798689.7213932971
798689.7213932971
Loss: 798689.7213932971
Loss: 1038385.382587344
Gradient: [-200.32072806   98.21596442 -417.14835494 ... -182.34423993 -814.78446007
  770.62001375]
Loss: 863211.6492981167
Loss: 798689.7213932971
Loss: 863211.6492981167
863211.6492981167
Loss: 863211.6492981167
Loss

In [13]:
idx = np.argpartition(w, -50)[-50:]
output = list()

for key, value in localDict.items():
    if value in idx:
        output.append(key)
print(output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['that', 'not', 'any', 'court', 'mr', 'act', 'evidence', 'decision', 'whether', 'applicant', 'application', 'tribunal', 'claim', 'costs', 'matter', 'reasons', 'ltd', 'appeal', 'respondent', 'orders', 'respect', 'relation', 'relevant', 'appellant', 'sought', 'notice', 'circumstances', 'hearing', 'proceedings', 'matters', 'consider', 'pty', 'respondents', 'proceeding', 'regard', 'judgment', 'satisfied', 'submissions', 'affidavit', 'pursuant', 'fca', 'clr', 'relied', 'hca', 'discretion', 'fcr', 'alr', 'fcafc', 'relevantly', 'gummow']

## Task 3

In [14]:
# Testing Data
import re
import numpy as np

# load up the small dataset
corpus = sc.textFile("s3://chrisjermainebucket/comp330_A5/TestingDataOneLinePerDoc.txt")
# each entry in validLines will be a line from the text file
validLines = corpus.filter(lambda x: 'id' in x)
# now we transform it into a bunch of (docID, text) pairs
keyAndText = validLines.map(lambda x: (x[x.index('id="') + 4:x.index('" url=')], x[x.index('">') + 2:]))

# TFIDF for all dictionaries
testingData = keyAndText.map(lambda x: (1 if x[0][0:2] == "AU" else 0, predictLabel(x[1])))
testingData = testingData.map(lambda x: (x[0], x[1][0]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Make predictions using all of the data points in x
# print ‘success’ or ‘failure’ depending on whether the 
# prediction is correct 

# x is the data set
# y is the labels
# w is the current set of weights

def predict(x, y, w):
    if((np.dot(x, w) > 0.008) and (y == 1)):
        # print("True Positive")
        return "tp"
    elif((np.dot(x, w) < 0.008) and (y == 0)):
        # print("True Negative")
        return "tn"
    elif((np.dot(x, w) > 0.008) and (y == 0)):
        # print("False Positive")
        return "fp"
    elif((np.dot(x, w) < 0.008) and (y == 1)):
        # print("False Negative")
        return "fn"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
prediction = testingData.map(lambda x: predict(x[1], x[0], w)).collect()

fpList = list()
tp = 0
tn = 0
fp = 0
fn = 0

for idx, i in enumerate(prediction):
    if i == "tp":
        tp += 1
    elif i == "tn":
        tn += 1
    elif i == "fp":
        fp += 1
        fpList.append(idx)
    elif i == "fn":
        fn += 1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
print(f'TP: {tp}')
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TP: 328
TN: 18340
FP: 7
FN: 49

In [38]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1score = 2 * precision * recall / (precision + recall)
print("%d out of %d correct." % (tp + tn, len(prediction)))
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1score}\n")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

18668 out of 18724 correct.
Precision: 0.9791044776119403
Recall: 0.870026525198939
F1 Score: 0.9213483146067415

In [39]:
print(fpList)
checkFP = keyAndText.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1617, 3342, 12672, 13317, 14579, 14610, 17997]

In [43]:
checkFP[13317]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('3279480', ' Court of Appeal of Singapore  The Court of Appeal of the Republic of Singapore is the nation\'s highest court and its court of final appeal. It is the upper division of the Supreme Court of Singapore, the lower being the High Court. The Court of Appeal consists of the Chief Justice of Singapore, who is the President of the Court, and the Judges of Appeal. The Chief Justice may ask judges of the High Court to sit as members of the Court of Appeal to hear particular cases. The seat of the Court of Appeal is the Supreme Court Building. The Court exercises only appellate jurisdiction in civil and criminal matters. In other words, it possesses no original jurisdiction\xa0– it does not deal with trials of matters coming before the court for the first time. In general, the Court hears civil appeals from decisions of the High Court made in the exercise of the latter\'s original and appellate jurisdiction, that is, decisions on cases that started in the High Court as well as decis